# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## English - Tigrinya

### Data
A mix of corpora is used: 
- JW300
- Tatoeba
- [Parallel Corpora for Ethiopian Languages](https://github.com/AAUThematic4LT/Parallel-Corpora-for-Ethiopian-Languages)

Test set is retrieved from masakhane global JW300 test set. 

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ti" 
lc = True  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "tigmix" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [3]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-ti-tigmix


In [4]:
# Install opus-tools
! pip install opustools-pkg

In [14]:
# Downloadi and extract JW300
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q
! gunzip JW300_latest_xml_$src-$tgt.xml.gz

# Downloading other corpora
# OPUS Tatoeba
! wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/en-ti.txt.zip
! unzip en-ti.txt.zip
! rm *.zip LICENSE README *.xml

# Ethiopian languages corpus
#! git clone https://github.com/AAUThematic4LT/Parallel-Corpora-for-Ethiopian-Languages.git
! cat Parallel-Corpora-for-Ethiopian-Languages/Exp\ I-English\ to\ Local\ Lang/Legal/tig_eng/other.eg > ethiopian.en
! cat Parallel-Corpora-for-Ethiopian-Languages/Exp\ I-English\ to\ Local\ Lang/Legal/tig_eng/other.tg > ethiopian.ti
! cat Parallel-Corpora-for-Ethiopian-Languages/Exp\ I-English\ to\ Local\ Lang/Legal/tig_eng/p_eng_et.txt >> ethiopian.en
! cat Parallel-Corpora-for-Ethiopian-Languages/Exp\ I-English\ to\ Local\ Lang/Legal/tig_eng/p_tig_et.txt >> ethiopian.ti
! cat Parallel-Corpora-for-Ethiopian-Languages/Exp\ I-English\ to\ Local\ Lang/Spritual/jw_bible/tig_eng/p_eng_et.txt >> ethiopian.en
! cat Parallel-Corpora-for-Ethiopian-Languages/Exp\ I-English\ to\ Local\ Lang/Spritual/jw_bible/tig_eng/p_tig_et.txt >> ethiopian.ti
! cat Parallel-Corpora-for-Ethiopian-Languages/Exp\ I-English\ to\ Local\ Lang/Spritual/jw_daily/tig_eng/p_eng_et >> ethiopian.en
! cat Parallel-Corpora-for-Ethiopian-Languages/Exp\ I-English\ to\ Local\ Lang/Spritual/jw_daily/tig_eng/p_tig_et >> ethiopian.ti

print("Total lines from Ethiopian corpus:")
! wc -l ethiopian.ti
! wc -l ethiopian.en

# Merge all corpora into one dataset: Tigmix
! cat *.en > tigmix.en
! cat *.ti > tigmix.ti

print("Total lines Tigmix:")
! wc -l tigmix.ti
! wc -l tigmix.en


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ti.xml.gz not found. The following files are available for downloading:

   4 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-ti.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  40 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/ti.zip

 307 MB Total size
./JW300_latest_xml_en-ti.xml.gz ... 100% of 4 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_ti.zip ... 100% of 40 MB
--2020-01-21 10:47:26--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/moses/en-ti.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9408 (9.2K) [application/zip]
Saving to: ‘en-ti.txt.zip’

en-ti.txt.zip       100%[===================>]   9.19K  --.-KB/s    in 0s      

2020-01-21 10:47:27 (319 MB/s) - ‘en-ti.txt.zip’ saved [9408/94

In [15]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-01-21 10:49:40--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.04s   

2020-01-21 10:49:40 (6.62 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-01-21 10:49:41--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-ti.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [16]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [20]:
import pandas as pd

# TMX file to dataframe
source_file = 'tigmix.' + source_language
target_file = 'tigmix.' + target_language

source = []
target = []
no_skip_lines = 0  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f_src, open(target_file) as f_tgt:
    for line_src, line_tgt in zip(f_src, f_tgt):
        # Skip sentences that are empty and contained in the test set.
        if line_src.strip() not in en_test_sents and line_src.strip() and line_tgt.strip():
          source.append(line_src.strip())         
          target.append(line_tgt.strip())
        else:
          no_skip_lines += 1
    
print('Loaded data and skipped {}/{} lines either empty or contained in test set.'.format(no_skip_lines, i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 31869/435545 lines either empty or contained in test set.


,source_sentence,target_sentence
0,Health care for asylum seekers and refugees in...,ሓልዮት ጥዕና ንደለይቲ ዑቕባን ስደተኛታትን ኣብ ስኮትላንድ
1,"In Scotland , most health care is provided by ...",መብዛሕትኡ ክንክን ጥዕና ኣብ ስኮትላንድ ብሃገራዊ ኣገልግሎት ጥዕና ( ሃ...
2,Everyone who lives legally in Scotland has a r...,ሕጋዊያን ነበርቲ ስኮትላንድ ፤ ዜግነቶም ብዘየገድስ መሰል ናይ ሃ.ኣ.ጥ....


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [28]:
print("Current corpus size: %i"%(len(df)))

# drop duplicate translations
df_pp = df.drop_duplicates()
print("After drop duplicates: %i"%len(df_pp))

#drop non-geez lines from target and geez lines in source
df_pp = df_pp[df_pp.target_sentence.str.contains('[\u1200-\u137f|\u1380-\u1394|\u2d80-\u2ddf|\uab00-\uab2f]',case=False)]
print("After drop non-geez on target: %i"%len(df_pp))
df_pp = df_pp[-df_pp.source_sentence.str.contains('[\u1200-\u137f|\u1380-\u1394|\u2d80-\u2ddf|\uab00-\uab2f]',case=False)]
print("After drop geez on source: %i"%len(df_pp))

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
# df_pp.drop_duplicates(subset='source_sentence', inplace=True)
# df_pp.drop_duplicates(subset='target_sentence', inplace=True)
# print("After drop conflicts: %i"%len(df_pp))

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

Current corpus size: 403677
After drop duplicates: 384332
After drop non-geez on target: 381449
After drop geez on source: 381350


In [29]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 2.1MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144672 sha256=9797a4dd64722f4a7f909ddc355dd961c72cf3a5ade89720476211003ba9713a
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.11 0.00 percent complete
00:00:18.99 0.26 percent complete
00:00:39.53 0.52 percent complete
00:00:59.44 0.79 percent complete
00:01:18.94 1.05 percent complete
00:01:39.10 1.31 percent complete
00:01:58.76 1.57 percent complete
00:02:18.27 1.84 percent complete
00:02:38.05 2.10 percent complete
00:02:57.15 2.36 percent complete
00:03:16.39 2.62 percent complete
00:03:37.22 2.88 percent complete
00:03:57.15 3.15 percent complete
00:04:16.54 3.41 percent complete
00:04:37.01 3.67 percent complete
00:04:56.55 3.93 percent complete
00:05:15.93 4.20 percent complete
00:05:35.88 4.46 percent complete
00

00:09:12.12 7.34 percent complete
00:09:31.16 7.60 percent complete
00:09:51.14 7.87 percent complete
00:10:10.42 8.13 percent complete
00:10:30.60 8.39 percent complete
00:10:49.70 8.65 percent complete
00:11:08.90 8.92 percent complete
00:11:28.76 9.18 percent complete
00:11:48.24 9.44 percent complete
00:12:07.35 9.70 percent complete
00:12:26.81 9.96 percent complete
00:12:46.85 10.23 percent complete
00:13:05.54 10.49 percent complete
00:13:24.39 10.75 percent complete
00:13:43.78 11.01 percent complete
00:14:03.56 11.28 percent complete
00:14:23.03 11.54 percent complete
00:14:42.33 11.80 percent complete
00:15:01.12 12.06 percent complete
00:15:20.67 12.32 percent complete
00:15:39.32 12.59 percent complete
00:15:58.35 12.85 percent complete
00:16:17.89 13.11 percent complete
00:16:37.35 13.37 percent complete
00:16:57.24 13.64 percent complete
00:17:16.98 13.90 percent complete
00:17:35.67 14.16 percent complete
00:17:55.11 14.42 percent complete
00:18:14.52 14.68 percent compl

00:23:08.66 18.62 percent complete
00:23:28.63 18.88 percent complete
00:23:47.25 19.14 percent complete


00:24:06.11 19.40 percent complete
00:24:24.76 19.67 percent complete
00:24:45.00 19.93 percent complete
00:25:04.04 20.19 percent complete
00:25:22.89 20.45 percent complete
00:25:42.27 20.72 percent complete
00:26:01.41 20.98 percent complete
00:26:20.61 21.24 percent complete
00:26:39.83 21.50 percent complete
00:26:59.22 21.76 percent complete
00:27:18.67 22.03 percent complete
00:27:38.12 22.29 percent complete
00:27:57.65 22.55 percent complete
00:28:16.51 22.81 percent complete
00:28:36.11 23.08 percent complete
00:28:55.73 23.34 percent complete
00:29:14.86 23.60 percent complete
00:29:34.88 23.86 percent complete
00:29:54.75 24.12 percent complete
00:30:13.80 24.39 percent complete
00:30:33.49 24.65 percent complete
00:30:52.22 24.91 percent complete
00:31:11.17 25.17 percent complete
00:31:31.01 25.44 percent complete
00:31:49.62 25.70 percent complete
00:32:07.96 25.96 percent complete
00:32:27.45 26.22 percent complete
00:32:47.14 26.48 percent complete
00:33:06.10 26.75 pe

00:48:13.98 39.07 percent complete
00:48:32.86 39.33 percent complete
00:48:51.66 39.60 percent complete


...

01:23:55.83 68.44 percent complete
01:24:15.81 68.70 percent complete
01:24:34.83 68.97 percent complete
01:24:54.46 69.23 percent complete
01:25:13.82 69.49 percent complete
01:25:32.94 69.75 percent complete
01:25:52.02 70.01 percent complete
01:26:11.45 70.28 percent complete
01:26:30.43 70.54 percent complete
01:26:49.54 70.80 percent complete
01:27:08.28 71.06 percent complete
01:27:27.16 71.33 percent complete
01:27:46.38 71.59 percent complete
01:28:05.35 71.85 percent complete
01:28:23.78 72.11 percent complete
01:28:43.70 72.37 percent complete
01:29:02.88 72.64 percent complete
01:29:22.67 72.90 percent complete
01:29:42.65 73.16 percent complete
01:30:01.59 73.42 percent complete


01:30:21.34 73.69 percent complete
01:30:40.11 73.95 percent complete
01:30:59.45 74.21 percent complete
01:31:18.74 74.47 percent complete
01:31:38.83 74.73 percent complete
01:31:58.21 75.00 percent complete
01:32:16.49 75.26 percent complete
01:32:35.97 75.52 percent complete
01:32:54.51 75.78 percent complete
01:33:13.40 76.05 percent complete
01:33:32.65 76.31 percent complete
01:33:51.34 76.57 percent complete
01:34:09.97 76.83 percent complete
01:34:30.00 77.09 percent complete
01:34:48.66 77.36 percent complete
01:35:07.42 77.62 percent complete
01:35:26.47 77.88 percent complete
01:35:45.41 78.14 percent complete
01:36:04.79 78.41 percent complete
01:36:23.74 78.67 percent complete
01:36:43.32 78.93 percent complete
01:37:02.92 79.19 percent complete
01:37:22.53 79.45 percent complete
01:37:41.91 79.72 percent complete
01:38:00.18 79.98 percent complete
01:38:19.22 80.24 percent complete
01:38:38.67 80.50 percent complete
01:38:57.91 80.77 percent complete
01:39:16.83 81.03 pe

01:43:09.27 84.17 percent complete
01:43:28.41 84.44 percent complete
01:43:46.77 84.70 percent complete
01:44:05.03 84.96 percent complete
01:44:23.98 85.22 percent complete


01:44:44.11 85.49 percent complete
01:45:03.60 85.75 percent complete
01:45:22.51 86.01 percent complete
01:45:42.00 86.27 percent complete
01:46:00.42 86.53 percent complete
01:46:18.52 86.80 percent complete
01:46:37.98 87.06 percent complete
01:46:56.79 87.32 percent complete
01:47:15.61 87.58 percent complete
01:47:35.69 87.85 percent complete
01:47:54.51 88.11 percent complete
01:48:13.05 88.37 percent complete
01:48:31.78 88.63 percent complete
01:48:50.60 88.89 percent complete
01:49:10.00 89.16 percent complete
01:49:28.75 89.42 percent complete
01:49:48.13 89.68 percent complete
01:50:07.42 89.94 percent complete
01:50:26.81 90.21 percent complete
01:50:46.23 90.47 percent complete
01:51:04.90 90.73 percent complete
01:51:23.89 90.99 percent complete
01:51:43.02 91.25 percent complete
01:52:01.42 91.52 percent complete
01:52:20.45 91.78 percent complete
01:52:40.07 92.04 percent complete
01:52:59.13 92.30 percent complete
01:53:18.26 92.57 percent complete
01:53:37.68 92.83 pe

02:00:18.98 98.33 percent complete
02:00:37.74 98.60 percent complete
02:00:56.97 98.86 percent complete
02:01:15.57 99.12 percent complete
02:01:34.42 99.38 percent complete
02:01:52.82 99.65 percent complete
02:02:11.54 99.91 percent complete


In [31]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
  df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
  df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
christian elders train younger men in the congregation
"in japan , for example , taro had from childhood centered his life on being loyal and obedient to his parents ."
consider what the bible teaches .
"[ map on page 8 , 9 ]"
true friendship is a wonderful gift from jehovah god .
7 : 12 .
"the apostle john wrote : “ we know we originate with god , but the whole world is lying in the power of the wicked one . ”"
"three years later , i was baptized ."
it makes me feel sick .
this has occurred in “ the time of the end ” ​ — the time in which we are now living .

==> train.ti <==
ክርስትያን ሽማግለታት ነቶም ኣብ ጉባኤ ዘለዉ መንእሰያት የሰልጥኑ
ንኣብነት ፡ ኣብ ጃፓን ዚነብር ታሮ ኻብ ንእስነቱ ኣትሒዙ ንወለዱ እሙንን እዙዝን እዩ ነይሩ ።
መጽሓፍ ቅዱስ እንታይ ከም ዚምህር እስከ ንርአ ።
"[ ኣብ ገጽ 8 , 9 ዘሎ ካርታ ]"
ሓቀኛ ዕርክነት ዘደንቕ ዝዀነ ውህበት የሆዋ ኣምላኽ ኢዩ ።
7 : 12 ።
ሃዋርያ ዮሃንስ “ ካብ ኣምላኽ ምዃንና : ብዘላ ዓለምውን በቲ እኩይ ተትሒዛ ምህላዋ ንፈልጥ ኢና ” ኢሉ ጸሓፈ ።
ድሕሪ ሰለስተ ዓመት ድማ ተጠመቕኩ ።
ቅጭ እዩ ዜምጽኣለይ ።
እዚ ድማ ኣብዚ እንነብረሉ ዘሎና “ ዘመን መወዳእታ ” ኢዩ ተፈጺሙ ።
==> dev.en <==
"long before jesus c



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [33]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 2204 (delta 8), reused 5 (delta 3), pack-reused 2184
Receiving objects: 100% (2204/2204), 2.60 MiB | 16.87 MiB/s, done.
Resolving deltas: 100% (1529/1529), done.
Processing /content/joeynmt
     |████████████████████████████████| 276kB 9.9MB/s 
     |████████████████████████████████| 307kB 62.2MB/s 
     |████████████████████████████████| 215kB 73.1MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 747kB 61.5MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=67549da08a638586439b5c2f3d159e654db46437decdb3c467bff46c9c19fe8d
  Stored in directory: /tmp/pip-ephem-wheel-cache-gr006ni6/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyam

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [34]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.ti     test.ti	   train.en
dev.bpe.en	dev.ti	     test.en	     train.bpe.en  train.ti
dev.bpe.ti	test.bpe.en  test.en-any.en  train.bpe.ti
bpe.codes.4000	dev.en	     test.bpe.ti     test.ti	   train.en
dev.bpe.en	dev.ti	     test.en	     train.bpe.en  train.ti
dev.bpe.ti	test.bpe.en  test.en-any.en  train.bpe.ti
BPE Xhosa Sentences
ዓብዪ ዋ@@ ል@@ ታ እምነት ( ሕ@@ ጡ@@ ብ ጽሑፍ 12 - 14 ርአ )
ናይ ድሕ@@ ነት ቍ@@ ራ@@ ዕ ርእሲ ( ሕ@@ ጡ@@ ብ ጽሑፍ 15 - 18 ርአ )
ሰባት ን@@ መጽሓፍ ቅዱስ ከም እት@@ ፈት@@ ዎን ንምሕ@@ ጋ@@ ዞም ዚ@@ ከ@@ ኣለ@@ ካ ት@@ ገብር ኸም ዘለ@@ ኻን ምስ ረኣ@@ ዩ ፡ ጽቡቕ ምላሽ ይ@@ ህ@@ ቡ እዮም ” ይብል ።
ናይ መንፈስ ሰይ@@ ፊ ( ሕ@@ ጡ@@ ብ ጽሑፍ 19 - 20 ርአ )
ብሓ@@ ገ@@ ዝ የሆዋ ፡ ክን@@ ቃ@@ ወ@@ ሞ ንኽእል ኢና !
Combined BPE Vocab
ù
@
➌
ā
ź
#
ʺ@@
ዃ
ṭ@@
እቆ@@


In [0]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 70                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8             # Increased to 8 for larger data.
        embeddings:
            embedding_dim: 512   # Increased to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # Increased to 512 for larger data.
        ff_size: 2048            # Increased to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8              # Increased to 8 for larger data.
        embeddings:
            embedding_dim: 512    # Increased to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 512         # Increased to 512 for larger data.
        ff_size: 2048            # Increased to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [36]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-01-21 13:43:24,056 Hello! This is Joey-NMT.
2020-01-21 13:43:25,233 Total params: 46600192
2020-01-21 13:43:25,234 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [0]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

In [0]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"